In [59]:
# the class to hande histograms and some helper functions
import copy
import math

def safediv(a,b):
    if float(b) != 0.:
        return a/b
    else:
        return 0.

def get_histo_from_file(fname, hname, htype):
    content = open(fname).read()
    found = False
    xmin = []
    xmax = []
    y = []
    yerr = []
    
    for l in content.split('\n'):
        if l.startswith("<histogram>") and hname in l and \
        ('TYPE@%s|' % htype in l or 'TYPE@%s"' % htype in l):
            found = True
            continue
        if found and l.startswith("<\\histogram>"):
            break
        if found:
            values = [float(v) for v in l.split()]
            xmin.append(values[0])
            xmax.append(values[1])
            y.append(values[2])
            yerr.append(values[3])
    if not found:
        raise Exception('get_histo_from_file, not found')
    return Histogram(xmin, xmax, y, yerr)
            
    
class Histogram(dict):
    def __init__(self, xmin, xmax, y, yerr):
        if not (len(xmin)==len(xmax)==len(y)==len(yerr)):
            raise Exception('Init, Invalid size')
                
        self['xmin'] = xmin
        self['xmax'] = xmax
        self['x'] = [(x1+x2)/2 for x1,x2 in zip(xmin,xmax)]
        self['y'] = y
        self['yerr'] = yerr
        self['nbins'] = len(xmin)
    
    def __add__(self, other):
        if not (len(self['y']) == len(other['y'])):
            raise Exception('add, Invalid size')
        if not (self['xmin'] == other['xmin'] and self['xmax'] == other['xmax']):
            raise Exception('add, Invalid x')
        return Histogram(self['xmin'], self['xmax'],
                        [y1+y2 for y1,y2 in zip(self['y'], other['y'])],
                        [math.sqrt(y1**2+y2**2) for y1,y2 in zip(self['yerr'], other['yerr'])])
    
    def __mul__(self, other):
        if type(other) == float or type(other) == int:
            return Histogram(self['xmin'], self['xmax'],
                    [y1*other for y1 in self['y']],
                        [y1*other for y1 in self['yerr']])
        else:
            raise Exception('mul, not implemented')
            
            
    def __truediv__(self, other):
        if type(other) == float or type(other) == int:
            return Histogram(self['xmin'], self['xmax'],
                    [y1/other for y1 in self['y']],
                        [y1/other for y1 in self['yerr']])
        elif type(other) == Histogram:
            if not (len(self['y']) == len(other['y'])):
                raise Exception('div, Invalid size')
            if not (self['xmin'] == other['xmin'] and self['xmax'] == other['xmax']):
                raise Exception('div, Invalid x')
            return Histogram(self['xmin'], self['xmax'],
                        [safediv(y1,y2) for y1,y2 in zip(self['y'], other['y'])],
                        [math.sqrt(safediv(e1,y2)**2+(safediv(y1,y2**2)*e2)**2) for y1,y2,e1,e2 in zip(self['y'], other['y'], self['yerr'], other['yerr'])])
            
        else:
            raise Exception('div, not implemented')

        

In [ ]:
# observables and histograms
import os 
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.lines as mlines
lw = 1.5
allcolors = list(mcolors.TABLEAU_COLORS.values()) + list(mcolors.BASE_COLORS.values()) + list(mcolors.XKCD_COLORS.values())
allcolors[1] = 'black'
allcolors[0] = 'blue'
allcolors[3] = 'red'
import matplotlib.font_manager as font_manager
import matplotlib.backends.backend_pdf as backend_pdf
from matplotlib.ticker import MultipleLocator,AutoMinorLocator
 
import matplotlib as mpl
#mpl.rcParams.update({"mathtext.fontset": "dejavuserif"})
mpl.rcParams.update({"text.usetex": True, "font.family": "serif", "font.size": 14})
mpl.rcParams.update({'lines.markersize': 8})
markers_glob = ['o', 'v', 's', '<', '>', 's', 'p', '*', 'h', 'p', 'x', 'X', 'd']
font_legend = font_manager.FontProperties(size=12)

def update_legend(ax, ncol):
    # this is to have the line in the legend, and not the square 
    handles, labels = ax.get_legend_handles_labels()
    new_handles = []
    for i, h in enumerate(handles):
        ls = h.get_linestyle()
        lc = h.get_edgecolor()
        #mk = markers[i]
        if ls == 'dashed':
            facecolor = 'none'
        else:
            facecolor = lc
        #new_handles.append(mlines.Line2D([], [], c=lc, linestyle=ls, marker = mk, markerfacecolor = facecolor))
        new_handles.append(mlines.Line2D([], [], c=lc, linestyle=ls, markerfacecolor = facecolor))

    ax.legend(handles=new_handles, labels=labels, prop=font_legend, frameon = False, ncol=ncol)



energies = [1,3,10]
observables = ['mtt']
processes = ['tt', 'ww']

xlabels = {'tt': {'mtt': 'm_{t\\bar t}'}, 
           'ww': {'mtt': 'm_{W^+W^-}'}}
proc_string = {'tt': '$\\mu^+\\mu^- \\to t \\bar t$',
               'ww': '$\\mu^+\\mu^- \\to W^+ W^-$'}
for p in processes:
    for en in energies:
        fname_mg = os.path.join(p, 'run_%dtev_NLO' % en, 'MADatNLO.HwU')
        fname_nnlo = os.path.join(p, 'run_nnlo_%d' % en, 'MADatNLO.HwU')
        for o in observables:
            # mg histograms
            h_lo_mg = get_histo_from_file(fname_mg, o, 'LO')
            h_loaa_mg = get_histo_from_file(fname_mg, o, 'LOAA')
            h_nlo_mg = get_histo_from_file(fname_mg, o, 'NLO')
            h_tot_mg = get_histo_from_file(fname_mg, o, 'TOT')
            # nnlo histograms
            h_lo_nnlo = get_histo_from_file(fname_nnlo, o, 'LO')
            h_nlo_nnlo = get_histo_from_file(fname_nnlo, o, 'NLO')
            h_nnlo_nnlo = get_histo_from_file(fname_nnlo, o, 'NNLO')
            h_tota = h_nnlo_nnlo + h_tot_mg
            
            # now the figure
            fig, axes = plt.subplots(nrows = 3, ncols= 1, sharex = True, 
                                     gridspec_kw={'height_ratios': [2.5,1,1]})
            fig.set_size_inches(7*0.75,7)
            fig.suptitle("\n\n" + "    " + proc_string[p] + 'at %d TeV' % en, fontsize=16, y=1.1 ) 
            
            for ax in axes:
                pos = ax.get_position()
                ax.set_position([pos.x0, pos.y0, pos.width*1.1, pos.height*1.1])
                ax.yaxis.set_minor_locator(AutoMinorLocator())
            
            #main frame, show absolute xsec
            ax = axes[0]
            ax.set_ylabel("$\sigma$ per bin [pb]")
            ax.set_yscale('log')
            if o == 'mtt' and p == 'tt':
                ax.set_xlim(340,en*1000)
            if o == 'mtt' and p == 'ww':
                ax.set_xlim(160,en*1000)
            # total
            ax.hist(x = h_tota['x'], weights = h_tota['y'], histtype= 'step', bins = h_tota['nbins'], 
                   label = '$\\textrm{LO}+\\textrm{NLO}+\\textrm{NNLO}^{\\gamma\\gamma}_\\textrm{coll}$',
                   linewidth=lw, color=allcolors[1])
            
            ax.hist(x = h_tot_mg['x'], weights = h_tot_mg['y'], histtype= 'step', bins = h_tot_mg['nbins'], 
                   label = '$\\textrm{LO}+\\textrm{NLO}$',
                   linewidth=lw, color=allcolors[2], linestyle='dashed')
            
            ax.hist(x = h_lo_mg['x'], weights = h_lo_mg['y'], histtype= 'step', bins = h_lo_mg['nbins'], 
                   label = '$\\textrm{LO}$',
                   linewidth=lw, color=allcolors[3], linestyle='dotted')
            
            ax.hist(x = h_loaa_mg['x'], weights = h_loaa_mg['y'], histtype= 'step', bins = h_loaa_mg['nbins'], 
                   label = '$\\textrm{LO}^{\\gamma\\gamma}$',
                   linewidth=lw, color=allcolors[4], linestyle='-.')
            
            ax.set_ylim(min([v for v in h_tota['y'] if v !=0])/2, max(h_tota['y'])*2)
            update_legend(ax, ncol=2)

            # inset, lo gamma gamma / lo total
            ax=axes[1]
            ax.set_ylabel("$\\textrm{LO}^{\\gamma\\gamma}/\\textrm{LO}$")
            
            h_loratio = h_loaa_mg / h_lo_mg
            ax.hist(x = h_loratio['x'], weights = h_loratio['y'], histtype= 'step', bins = h_loratio['nbins'], 
                   linewidth=lw, color=allcolors[4], linestyle='-.')
            
            
            ax=axes[2]
            ax.set_ylabel("Ratio over LO")
            ax.hist(x = h_lo_mg['x'], weights = [0]*h_lo_mg['nbins'], histtype= 'step', bins = h_kfac['nbins'], 
                   linewidth=lw/2, color='gray', linestyle='solid',)        
    
            h_kfac = h_nlo_mg / h_lo_mg
            ax.hist(x = h_kfac['x'], weights = h_kfac['y'], histtype= 'step', bins = h_kfac['nbins'], 
                   linewidth=lw, color=allcolors[2], linestyle='dashed', 
                    label = '$\\textrm{NLO}$')
            
            h_kfac = h_nlo_nnlo / h_lo_mg
            ax.hist(x = h_kfac['x'], weights = h_kfac['y'], histtype= 'step', bins = h_kfac['nbins'], 
                   linewidth=lw, color=allcolors[10], linestyle='dotted', 
                    label = '$\\textrm{NLO}^{\\gamma\\gamma}_\\textrm{coll}$')
            
            h_kfac = h_nnlo_nnlo / h_lo_mg * 2
            ax.hist(x = h_kfac['x'], weights = h_kfac['y'], histtype= 'step', bins = h_kfac['nbins'], 
                   linewidth=lw, color=allcolors[1], linestyle='solid', 
                    label = '$\\textrm{NNLO}^{\\gamma\\gamma}_\\textrm{coll} \\times 2$')
            update_legend(ax, ncol=3)
            ax.set_xlabel('$%s$ [GeV]' % xlabels[p][o])
            
            plt.savefig('%s_%s_%dtev.pdf' % (o,p,en), bbox_inches = 'tight')
            
 
    